In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",  # use a supported model
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke("The first person to land on the moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [14]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.smartrecruiters.com/Sutherland/744000084572070")
page_data = loader.load().pop().page_content
print(page_data)

Sutherland Associate - Account Management | SmartRecruiters


















Google Chrome
Microsoft Edge
Apple Safari
Mozilla Firefox









Associate - Account ManagementFull-timeCompany DescriptionAbout SutherlandArtificial Intelligence. Automation. Cloud engineering. Advanced analytics. For business leaders, these are key factors of success. For us, they’re our core expertise.
We work with iconic brands worldwide. We bring them a unique value proposition through market-leading technology and business process excellence.We’ve created over 200 unique inventions under several patents across AI and other critical technologies. Leveraging our advanced products and platforms, we drive digital transformation, optimize critical business operations, reinvent experiences, and pioneer new solutions, all provided through a seamless “as a service” model.For each company, we provide new keys for their businesses, the people they work with, and the customers they serve. We tailor proven and ra

In [16]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scrapped text is from the carrer's page of a website.
    Your job is to extract the job postings and return them in JSON Format containing the
    following keys :
    'role','experience','skills',and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Associate - Account Management",
  "experience": "0-1 year (2025 pass-outs can apply)",
  "skills": [
    "Excellent verbal and written English communication skills",
    "Good analytical and typing skills",
    "Flexibility to work in shifts, including night shifts"
  ],
  "description": "The consultant is responsible for delivering an outstanding customer experience via email, chat, or social media by applying the appropriate principles to resolve any issues, inquiries, or concerns."
}
```


In [17]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [18]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Associate - Account Management',
 'experience': '0-1 year (2025 pass-outs can apply)',
 'skills': ['Excellent verbal and written English communication skills',
  'Good analytical and typing skills',
  'Flexibility to work in shifts, including night shifts'],
 'description': 'The consultant is responsible for delivering an outstanding customer experience via email, chat, or social media by applying the appropriate principles to resolve any issues, inquiries, or concerns.'}

In [19]:
type(json_res)

dict

In [20]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [21]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids =[str(uuid.uuid4())])

In [22]:
links = collection.query(query_texts=["Experience in python", "Experience in React"],n_results = 2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [23]:
job = json_res
job['skills']

['Excellent verbal and written English communication skills',
 'Good analytical and typing skills',
 'Flexibility to work in shifts, including night shifts']

In [24]:
prompt_email = PromptTemplate.from_template(
            """
            ### JOB DESCRIPTION:
            {job_description}

            ### INSTRUCTION:
            You are Tejeshwari, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools.

            Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.
            Your job is to write a concise cold email to the client regarding the job mentioned above, describing the capabilities of AtliQ and how we can help in fulfilling their needs.
            Also add the most relevant ones from the following links to showcase AtliQ's portfolio: {link_list}
            Remember you are Tejeshwari, BDE at AtliQ.
            Do not provide a preamble.
            ### EMAIL (NO PREAMBLE):
            """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Account Management Solutions with AtliQ

Dear Hiring Manager,

I came across the job description for an Associate - Account Management role at your esteemed organization and was impressed by the emphasis on delivering outstanding customer experiences. As a Business Development Executive at AtliQ, an AI & Software Consulting company, I believe our expertise can help you fulfill your account management needs.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team excels in developing automated tools that can enhance your customer experience via email, chat, or social media. With excellent verbal and written English communication skills, good analytical and typing skills, and flexibility to work in shifts, our consultants can seamlessly integrate with your team.

Our portfolio showcases our capabilities in developing innovative solu